In [1]:
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Date').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/16 18:30:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
l = [('X',)]

In [4]:
df = spark.createDataFrame(l).toDF('dummy')

In [5]:
df.show()

+-----+
|dummy|
+-----+
|    X|
+-----+



In [6]:
df.printSchema()

root
 |-- dummy: string (nullable = true)



In [7]:
df.select(current_date().alias('current date')).show()

+------------+
|current date|
+------------+
|  2023-08-16|
+------------+



In [8]:
df.select(current_timestamp().alias('current timestamp')).show(truncate = False)

+--------------------------+
|current timestamp         |
+--------------------------+
|2023-08-16 18:30:51.429877|
+--------------------------+



In [9]:
df.select(to_date(lit('20180101'),'yyyyMMdd').alias('date')).show()

+----------+
|      date|
+----------+
|2018-01-01|
+----------+



In [10]:
df.select(to_timestamp(lit('20181203 175043987'),'yyyyMMdd HHmmssSSS').alias('timestamp')).show(truncate = False)

+-----------------------+
|timestamp              |
+-----------------------+
|2018-12-03 17:50:43.987|
+-----------------------+



In [11]:
datetimes = [("2014-02-28", "2014-02-28 10:00:00.123"),
                     ("2016-02-29", "2016-02-29 08:08:08.999"),
                     ("2017-10-31", "2017-12-31 11:59:59.123"),
                     ("2019-11-30", "2019-08-31 00:00:00.000")
                ]

In [12]:
datetimeDF = spark.createDataFrame(datetimes,schema = """date STRING, timestamp STRING""")

In [13]:
datetimeDF.show(truncate = False)

+----------+-----------------------+
|date      |timestamp              |
+----------+-----------------------+
|2014-02-28|2014-02-28 10:00:00.123|
|2016-02-29|2016-02-29 08:08:08.999|
|2017-10-31|2017-12-31 11:59:59.123|
|2019-11-30|2019-08-31 00:00:00.000|
+----------+-----------------------+



In [14]:
datetimeDF.printSchema()

root
 |-- date: string (nullable = true)
 |-- timestamp: string (nullable = true)



In [16]:
datetimeDF.withColumn('Add date',date_add('date',10)). \
            withColumn('Add timestamp',date_add('timestamp',10)). \
            withColumn('Sub date',date_sub('date',10)). \
            withColumn('Sub timestamp',date_sub('date',10)).show(truncate = False)

+----------+-----------------------+----------+-------------+----------+-------------+
|date      |timestamp              |Add date  |Add timestamp|Sub date  |Sub timestamp|
+----------+-----------------------+----------+-------------+----------+-------------+
|2014-02-28|2014-02-28 10:00:00.123|2014-03-10|2014-03-10   |2014-02-18|2014-02-18   |
|2016-02-29|2016-02-29 08:08:08.999|2016-03-10|2016-03-10   |2016-02-19|2016-02-19   |
|2017-10-31|2017-12-31 11:59:59.123|2017-11-10|2018-01-10   |2017-10-21|2017-10-21   |
|2019-11-30|2019-08-31 00:00:00.000|2019-12-10|2019-09-10   |2019-11-20|2019-11-20   |
+----------+-----------------------+----------+-------------+----------+-------------+



In [17]:
datetimeDF.withColumn('datediff date',datediff(current_date(),'date')). \
            withColumn('datediff timestamp',datediff(current_timestamp(),'timestamp')). \
            show(truncate = False)

+----------+-----------------------+-------------+------------------+
|date      |timestamp              |datediff date|datediff timestamp|
+----------+-----------------------+-------------+------------------+
|2014-02-28|2014-02-28 10:00:00.123|3456         |3456              |
|2016-02-29|2016-02-29 08:08:08.999|2725         |2725              |
|2017-10-31|2017-12-31 11:59:59.123|2115         |2054              |
|2019-11-30|2019-08-31 00:00:00.000|1355         |1446              |
+----------+-----------------------+-------------+------------------+



In [24]:
datetimeDF.withColumn('add month date',add_months('date',3)). \
            withColumn('add month timestamp',add_months('timestamp',3)). \
            withColumn('month between date',round(months_between(current_date(),'date'),2)). \
            withColumn('month between timestamp',round(months_between(current_date(),'timestamp'),2)). \
            show()

+----------+--------------------+--------------+-------------------+------------------+-----------------------+
|      date|           timestamp|add month date|add month timestamp|month between date|month between timestamp|
+----------+--------------------+--------------+-------------------+------------------+-----------------------+
|2014-02-28|2014-02-28 10:00:...|    2014-05-28|         2014-05-28|            113.61|                  113.6|
|2016-02-29|2016-02-29 08:08:...|    2016-05-29|         2016-05-29|             89.58|                  89.57|
|2017-10-31|2017-12-31 11:59:...|    2018-01-31|         2018-03-31|             69.52|                   67.5|
|2019-11-30|2019-08-31 00:00:...|    2020-02-29|         2019-11-30|             44.55|                  47.52|
+----------+--------------------+--------------+-------------------+------------------+-----------------------+

